In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.preprocessing import StandardScaler
import folium
from folium.plugins import HeatMap


In [ ]:
df = pd.read_csv('/content/tow_coords.csv')

df = df.dropna(subset=['lat', 'lon'])

# Extract latitude and longitude
points = df[['lat', 'lon']].values

# Compute the Voronoi diagram
vor = Voronoi(points)

# Plot the Voronoi diagram
fig = voronoi_plot_2d(vor)
plt.title('Voronoi Diagram of Tow Coordinates')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
plt.scatter(df['lat'],df['lon'])

In [ ]:
data = pd.read_csv('/content/tow_coords.csv') # I know I know its same..
# Define the reference point as the first coordinate
lat_ref = data.loc[0, 'lat']
lon_ref = data.loc[0, 'lon']

# Function to convert lat/lon to xy coordinates relative to the reference point
def lat_lon_to_xy(lat, lon, lat_ref, lon_ref):
    # Radius of the Earth in meters
    R = 6371000

    # Convert degrees to radians
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    lat_ref_rad = np.radians(lat_ref)
    lon_ref_rad = np.radians(lon_ref)

    # Calculate x and y with positive coordinates (first quadrant)
    x = R * (lon_rad - lon_ref_rad) * np.cos(lat_ref_rad)
    y = R * (lat_rad - lat_ref_rad)

    # Ensure all coordinates are in the first quadrant by adding an offset
    x_offset = abs(x.min()) if x.min() < 0 else 0
    y_offset = abs(y.min()) if y.min() < 0 else 0

    x += x_offset
    y += y_offset

    return x, y

# Apply the function to the dataframe to calculate xy coordinates
data['x'], data['y'] = lat_lon_to_xy(data['lat'], data['lon'], lat_ref, lon_ref)

# Display the new dataframe with x and y coordinates
# Removed the line that uses 'ace_tools'
# Instead, just print the DataFrame
print("XY Coordinates from Latitude and Longitude:")
print(data) # Use print(data) to display the DataFrame directly

# Display the first few rows of the updated data to verify
data.head()

plt.scatter(data['x'], data['y'])
plt.xlabel('X')
plt.ylabel('Y')
plt.title('XY Coordinates')
plt.grid(True)
plt.show()

In [ ]:
# Calculate the min and max for lat and lon to identify the region with most data points
lat_min, lat_max = data['lat'].quantile(0.009), data['lat'].quantile(0.99)
lon_min, lon_max = data['lon'].quantile(0.009), data['lon'].quantile(0.99)

# Create a filtered dataset that falls within the desired range
filtered_data = data[(data['lat'] >= lat_min) & (data['lat'] <= lat_max) &
                     (data['lon'] >= lon_min) & (data['lon'] <= lon_max)]

# Plot the filtered latitude and longitude data as a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(filtered_data['lon'], filtered_data['lat'],marker='x', color='green', label='Filtered Coordinates')

# Label axes
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter Plot of Latitude and Longitude (Filtered Region)')

# Add a legend
plt.legend()

# Display the plot with restricted domain
plt.grid(True)
plt.show()


In [ ]:
# Manually filter the data to include only those points where latitude is less than 40 and longitude is greater than -77.1
filtered_data_manual = data[(data['lat'] < 40) & (data['lon'] > -77.1)]
#39.05, -77.10
# Plot the manually filtered latitude and longitude data as a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(filtered_data_manual['lon'], filtered_data_manual['lat'], color='purple', label='Filtered Coordinates (y < 40 and x > -77.1)')

# Label axes
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter Plot of Latitude and Longitude (Restricted y < 39.05 and x > -77.1)')

# Add a legend
plt.legend()

# Display the plot with the manually restricted domain
plt.grid(True)
plt.show()


In [ ]:
# Manually filter the data to include only those points where latitude is between 38.9 and 40, and longitude is greater than -77.1
filtered_data_manual = data[(data['lat'] > 38.9) & (data['lat'] < 40) & (data['lon'] > -77.1)]

# Plot the manually filtered latitude and longitude data as a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(filtered_data_manual['lon'], filtered_data_manual['lat'], color='orange', label='Filtered Coordinates (38.9 < y < 40 and x > -77.1)')

# Label axes
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter Plot of Latitude and Longitude (Restricted 38.9 < y < 40 and x > -77.1)')

# Add a legend
plt.legend()

# Display the plot with the manually restricted domain
plt.grid(True)
plt.show()


In [ ]:
data = pd.read_csv('/content/tow_coords.csv')

points_filtered = filtered_data[['lon', 'lat']].values

# Create a Voronoi diagram from the filtered points
vor_filtered = Voronoi(points_filtered)

# Plot the Voronoi diagram with a higher quality setting
plt.figure(figsize=(40, 30), dpi=500)  # Increase the figure size and resolution (DPI)
voronoi_plot_2d(vor_filtered, show_vertices=False, line_colors='blue', line_width=1.5, line_alpha=0.7, point_size=15)

# Overlay the filtered points to make it clear which points generate each Voronoi cell
plt.scatter(filtered_data['lon'], filtered_data['lat'], marker='x', color='green', s=5)

# Label axes
plt.xlabel('Longitude', fontsize=6)
plt.ylabel('Latitude', fontsize=6)
plt.title('Voronoi Diagram of Filtered Latitude and Longitude', fontsize=18)
plt.legend(fontsize=14)
plt.grid(True, linestyle='--', linewidth=0.5)
plt.show()


In [ ]:



# Assuming your data is in 'tow_coords.csv', adjust if needed
tow_coords_cleaned_df = pd.read_csv('/content/tow_coords.csv') # Load your data into tow_coords_cleaned_df


tow_coords_cleaned_df = tow_coords_cleaned_df.dropna(subset=['lat', 'lon'])

# Create a base map centered around the approximate center of Montgomery County, Maryland
map_center = [39.05, -77.10]  # Approximate center of Montgomery County
m = folium.Map(location=map_center, zoom_start=11)

# Extract coordinates to a list of [lat, lon] pairs for the heatmap
heat_data = tow_coords_cleaned_df[['lat', 'lon']].values.tolist()

# Add a HeatMap layer to the map
HeatMap(heat_data).add_to(m)

# Save the map to an HTML file to view
map_path = "tow_heatmap.html"
m.save(map_path)

# Display path for downloading
map_path

In [ ]:



# Load your median income dataset
income_df = pd.read_csv('/content/tow_coords_income.csv')

# Drop rows with missing latitude, longitude, or median income values
income_df = income_df.dropna(subset=['lat', 'lon', 'median_household_income'])

# Create a base map centered around the approximate center of Montgomery County, Maryland
map_center = [39.05, -77.10]  # Approximate center of Montgomery County
m_income = folium.Map(location=map_center, zoom_start=11)

# Extract coordinates and income as weighted data for the heatmap
# The HeatMap will be weighted by the median income to show intensity related to income levels
heat_data_income = income_df[['lat', 'lon', 'median_household_income']].values.tolist()

# Add a HeatMap layer to the map, using the median income as weight
HeatMap(heat_data_income, radius=10).add_to(m_income)

# Save the map to an HTML file to view
map_income_path = "income_heatmap.html"
m_income.save(map_income_path)

# Display path for downloading
print("Map saved at:", map_income_path)


In [ ]:
# Create a base map centered around Montgomery County, Maryland
m_combined = folium.Map(location=map_center, zoom_start=11)

# Add towing heatmap layer
towing_heat_layer = folium.FeatureGroup(name='Towing Hotspots')
HeatMap(heat_data).add_to(towing_heat_layer)
towing_heat_layer.add_to(m_combined)

# Add income heatmap layer
income_heat_layer = folium.FeatureGroup(name='Income Levels')
HeatMap(heat_data_income, radius=10).add_to(income_heat_layer)
income_heat_layer.add_to(m_combined)

# Add LayerControl to switch between layers
folium.LayerControl().add_to(m_combined)

# Save the map to an HTML file to view
m_combined.save("combined_towing_income_map.html")
